three types of dimension:
* plate dims
* sample dimensions (usually indexed K)
* user dims; underlying dimensions (which the user gets to interact with). 

In [340]:
import torch.nn as nn
from torch.distributions import Normal
import numpy as np

import sys; sys.path.append("..")
from tpp_trace import *
import utils as u
import tensor_ops as tpp

import imp
imp.reload(tpp)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Run

In [341]:
kappa = 3
n = 2
tr = sample_and_eval(chain_dist, draws=kappa, nProtected=n, data={"a": 2})

#tr = sample_and_eval(plate_chain_dist, draws=kappa, nProtected=2)#, data={"a": 2})
#tr.trace.out_dicts

tr.trace.out_dicts['sample']['__a']

tensor([[[ 2.0307,  2.2286,  1.5456]],

        [[-1.1578, -5.4063,  3.3367]],

        [[-0.7263,  4.6261, -0.4760]]], names=('_k__a', 'pos_A', 'pos_B'))

## index-aware summing

We have: one factor corresponding to each variable (latent or observed)

e.g. the trace output for the 4 gaussians in our chain example

every time we sample, we add a new dimension. Need to delete these after eval

### e.g.

In [69]:
#import imp
#imp.reload(tpp)
#imp.reload(u)

kappa = 2
n = 2
data = {} # {"a": [4] * 100}
tr = sample_and_eval(chain_dist, draws=kappa, nProtected=n, data=data)
tensors = tr.trace.out_dicts['log_prob']

X = t.Tensor([[.3,.1],\
              [.1,.3]])
X = X.refine_names('_k__a', '_k__b')
Y = t.Tensor([[.3,.7],\
              [.2,.3]])
Y = Y.refine_names('_k__a', '_k__b')




## 3 equivalent reductions

for this easy 2D case

In [81]:
# for every dim:
# log -> complicated index stuff -> sum all tensors -> logsumexp out dim
tensors = {'__a' : X.log(), \
           '__b' : Y.log() } 
TPP = tpp.combine_tensors(tensors)
TPP = list(TPP.values())[0]


# mean then sum on linear scale, then log result
logsummul = (X * Y / 4).sum().log()


# an addition operation in linear-scale becomes the LSE in log-scale. 
# mean -> log -> lse dims
dim = "_k__a"
logmean = (X * Y / 4).log()
lselogmul = logmean.logsumexp(dim) \
        .logsumexp("_k__b")


TPP, logsummul, lselogmul

(tensor(-2.6956), tensor(-2.6956), tensor(-2.6956))

In [99]:
T = t.Tensor([4,8])
np.log(4 + 8), T.log().logsumexp(0), T.sum(0).log()


(2.4849066497880004, tensor(2.4849), tensor(2.4849))

# Playing

In [119]:
# cf. docs
T = t.randn(1, 2, 3 , names=('N', 'C', '_K'))
T.names
#('N', 'C', 'H', 'W')

T3 = T.align_to('C', '_K', ...)

T2 = T.rename(_K='aaaaa')
T2.names #('N', 'C', 'height', 'width)

T, T.align_as(T3)


(tensor([[[-1.9652,  2.1366, -2.2147],
          [ 0.2629,  0.7617, -0.1724]]], names=('N', 'C', '_K')),
 tensor([[[-1.9652],
          [ 2.1366],
          [-2.2147]],
 
         [[ 0.2629],
          [ 0.7617],
          [-0.1724]]], names=('C', '_K', 'N')))

In [7]:
def named_example_2D() :
    X = t.Tensor([[.2,.1],\
                  [.1,.3]])
    X = X.refine_names('_k__a', '_k__b')
    Y = t.Tensor([[.9,.2],\
                  [.1,.1]])
    Y = Y.refine_names('_k__a', '_k__b')
    
    return X, Y, '_k__a'


def test_logmulmeanexp() :
    X, Y, dim = named_example_2D()
    log_mean_prod_exp = u.logmulmeanexp(X, Y, dim)
    
    reference = u.logmmmeanexp(X, Y) \
                .rename(None) \
                .diag()
    stripped = log_mean_prod_exp \
                .rename(None) \
                .squeeze()
    
    #assert(t.allclose(stripped, reference) )
    print(reference, "\n", stripped)
    
test_logmulmeanexp()

tensor([0.7480, 0.3512]) 
 tensor([0.7480, 0.3512])


tensor(0.5637)

In [ ]:
def rearrange_by_plate(tensor_dict) :
    """
    :param tensor_dict: dict of log_prob tensors
    :return: dict of dicts of log_prob tensors, dividing by plates
    """
    return NotImplementedError()


# Sum:  sum over each plate
def plate_sum() :
    return NotImplementedError()

In [ ]:
# Consider Figure 1 from TMC

"""
z2 | z1
z3 | z1
z4 | z2
x | z3, z4
"""
def simple_dist(trace):
    k1 = trace["i1"](WrappedDist(Normal, t.ones(3), 3))
    k2 = trace["i2"](WrappedDist(Normal, k1, 3))
    k3 = trace["i3"](WrappedDist(Normal, k2, 3))
    (k3,) = trace.delete_names(("i1", "i2"), (k3,))
    k4 = trace["i4"](WrappedDist(Normal, k3, 3))
    
    return k4

tr = sample_and_eval(simple_dist, draws=kappa, nProtected=2)#, data={"a": 2})

## index-aware sampling